<a href="https://colab.research.google.com/github/cbun1123/map_fit/blob/main/Stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!git clone https://github.com/cbun1123/map_fit

!unzip /content/map_fit/X_test_1.zip -d /content/map_fit/X_test
!unzip /content/map_fit/X_test_2.zip -d /content/map_fit/X_test
!unzip /content/map_fit/X_test_3.zip -d /content/map_fit/X_test

!rm /content/map_fit/*.zip

!pip install pytorch-lightning
%env PYTHONPATH=.:$PYTHONPATH

In [5]:
!nvidia-smi

Tue Jul 19 09:21:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import numpy as np
import cv2
import glob
import math

from google.colab.patches import cv2_imshow

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

import torchvision
from torchvision import transforms

In [20]:
filenames = glob.glob("/content/map_fit/X_test/*.jpg")
filenames.sort()
images = [cv2.imread(img) for img in filenames]

X_train = np.stack(images, axis=0)
y_train = np.load('/content/map_fit/Y_test.npy')
print(np.shape(X_train))

SIZE = 224

lin = np.linspace(-1,1,SIZE)
[Xm,Ym] = np.meshgrid(lin,lin)
idx = ((Xm**2+Ym**2)<1)
idx = np.stack([idx,idx,idx], axis=0)
idx_t = torch.from_numpy(idx)

class imageDataset(Dataset):
    def __init__(self):
        'Initialization'
        self.X = X_train
        self.y = y_train
        self.n_samples = self.X.shape[0]
        self.transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = self.X[index]
        X = self.transform(X)
        X = torch.mul(X,idx_t)

        y = self.y[index]
        y = torch.from_numpy(y)
        sample = X,y
        return sample

    def __len__(self):
        return self.n_samples


dataset = imageDataset()
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

(3000, 224, 224, 3)


In [21]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model init
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

# hyperparameters
num_epochs = 2
learning_rate = 0.01

# optimizer
optimizer = torch.optim.Adam(, lr=learning_rate)


total_samples = len(dataset)
n_iteration = math.ceil(total_samples/32)

for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(dataloader):
        # Trucs
        if (i % 10) == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iteration}, inputs {inputs.shape}')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch 1/2, step 1/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 11/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 21/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 31/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 41/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 51/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 61/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 71/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 81/94, inputs torch.Size([32, 3, 224, 224])
epoch 1/2, step 91/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 1/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 11/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 21/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 31/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 41/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 51/94, inputs torch.Size([32, 3, 224, 224])
epoch 2/2, step 61/94, inputs torch.Size([